In [1]:
# Importem les llibreries necessàries
import requests
import pandas as pd
import datetime
import plotly.express as px
import plotly

In [2]:
# Introduïm manualment al data que ens volem descarregar a l'API
data_manual = '11-09-2020'

# Convertim la data que hem introduït manualment al format necessari per a l'API
data_obj = datetime.datetime.strptime(data_manual, '%d-%m-%Y')
str_data = data_obj.strftime("%Y-%m-%dT%H:%M:%S.000")

In [ ]:
# Fem request i ens fem amb la informació del dia desitjat en format JSON
path_template = 'https://analisi.transparenciacatalunya.cat/resource/jj6z-iyrp.json?data={data}'
path = path_template.format(data=str_data)
json_request = requests.get(path).json()

# Convertim la resposta rebuda de JSON a dataframe amb pandas
df = pd.DataFrame.from_records(json_request)

In [4]:
# Mirem com ha quedat el dataframe que hem creat
df.head()

,comarcacodi,comarcadescripcio,data,municipicodi,municipidescripcio,numcasos,resultatcoviddescripcio,sexecodi,sexedescripcio
0,2.0,ALT EMPORDA,2020-09-11T00:00:00.000,17178.0,SANT PERE PESCADOR,2,Positiu PCR,0.0,Home
1,28.0,PLA DE L'ESTANY,2020-09-11T00:00:00.000,17015.0,BANYOLES,2,Positiu PCR,0.0,Home
2,36.0,TARRAGONES,2020-09-11T00:00:00.000,43905.0,SALOU,2,Positiu PCR,1.0,Dona
3,32.0,SEGARRA,2020-09-11T00:00:00.000,25072.0,CERVERA,6,Positiu PCR,1.0,Dona
4,13.0,BARCELONES,2020-09-11T00:00:00.000,8245.0,SANTA COLOMA DE GRAMENET,1,Positiu per ELISA,1.0,Dona


In [5]:
# El camp 'numcasos' s'ha de canviar de format, convertim de string a integer
df['numcasos'] = df['numcasos'].astype(int)

# Agrupem el dataframe per noms de comarca i municipi i sumaritzem els positius de COVID-19
df_agrupat = df.groupby(['comarcadescripcio','municipidescripcio']).sum().reset_index()

In [6]:
# Consultem finalment com ha quedat la taula amb les dades agregades per comarca i municipi
df_agrupat.head()

,comarcadescripcio,municipidescripcio,comarcacodi,municipicodi,numcasos,sexecodi
0,ALT EMPORDA,FIGUERES,2.0,17066.0,1,0.0
1,ALT EMPORDA,ROSES,2.0,17152.0,2,0.0
2,ALT EMPORDA,SANT PERE PESCADOR,2.0,17178.0,2,0.0
3,ALT EMPORDA,VENTALLÃ“,2.0,17210.0,1,1.0
4,ALT PENEDES,OLÃˆRDOLA,3.0,8145.0,1,0.0


In [ ]:
# Grafiquem  amb el treemap de Plotly, on agrupem els municipis per comarca
fig = px.treemap(df_agrupat,
                 path = ["comarcadescripcio","municipidescripcio"],
                 values = "numcasos",
                 color = 'comarcadescripcio',
                 width=1000, height=700,
                 title = "COVID-19 per comarques i municipis a " + data_manual,
                )

fig.show()

<img src="gencat_03.png">